<a href="https://colab.research.google.com/github/woneuy01/NLP_Jon/blob/master/sequence_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sequence Generation

Using an [example taken directly from The TensorFlow Authors](https://nbviewer.jupyter.org/github/tensorflow/docs/blob/master/site/en/tutorials/text/text_generation.ipynb), in this notebook we generate novel sequences of text based on a corpus of training data consisting of Shakespeare's complete works.

#### Import libraries

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import numpy as np
import os
import time

#### Download data

In [0]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

#### Import data

In [0]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [0]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [0]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

65 unique characters


#### Convert characters to numerical index representation

In [0]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [0]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '3' :   9,
  ':' :  10,
  ';' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,
  ...
}


In [0]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'First Citizen' ---- characters mapped to int ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


#### Create training examples and targets

In [0]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create stream of characters for training dataset
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

F
i
r
s
t


In [0]:
# Each sequence has 101 characters; we'll predict next char given the preceding ones, up to 100 chars of input
sequences = char_dataset.batch(seq_length+1, drop_remainder=True) 

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [0]:
for input_example, target_example in  dataset.take(1):
    print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [0]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 18 ('F')
  expected output: 47 ('i')
Step    1
  input: 47 ('i')
  expected output: 56 ('r')
Step    2
  input: 56 ('r')
  expected output: 57 ('s')
Step    3
  input: 57 ('s')
  expected output: 58 ('t')
Step    4
  input: 58 ('t')
  expected output: 1 (' ')


#### Shuffle data for stochastic gradient descent

In [0]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

#### Design neural network architecture

In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True, # last state for index i in batch is used as initial state in next batch
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ]) # we put all models in a list
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [0]:
# Examine the shape of the output
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [0]:
sampled_indices

array([41,  5, 11, 31, 38, 47, 14, 15, 56, 11,  2, 16, 61, 11, 34, 49,  0,
       56, 35, 57, 17,  7, 48, 51, 42, 49, 17, 25, 48, 52, 50, 41,  8, 60,
        0, 33, 47, 36, 42,  2, 53, 44,  5,  5, 52, 18, 34,  4, 20,  7, 16,
       64, 57, 22, 13, 32, 13, 24, 32, 51,  2, 11, 60, 13, 58, 51, 26, 20,
       14, 40, 59, 48, 47, 37, 50,  0, 39,  6, 63, 52, 36, 47,  4, 30, 33,
       45, 14, 15, 58, 13, 30, 53, 26, 10, 48, 25, 30, 50, 29, 59])

In [0]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 "st I did embrace him: yet his nature\nIn that's no changeling; and I must excuse\nWhat cannot be amend"

Next Char Predictions: 
 "c';SZiBCr;!Dw;Vk\nrWsE-jmdkEMjnlc.v\nUiXd!of''nFV&H-DzsJATALTm!;vAtmNHBbujiYl\na,ynXi&RUgBCtARoN:jMRlQu"


#### Compile model

In [0]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.173575


In [0]:
model.compile(optimizer='adam', loss=loss)

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './model_output/seqGen'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

#### Train!

In [0]:
EPOCHS=30

In [0]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
172/172 [==============================] - 152s 882ms/step - loss: 2.6992
Epoch 2/30
172/172 [==============================] - 150s 874ms/step - loss: 1.9659
Epoch 3/30
172/172 [==============================] - 150s 874ms/step - loss: 1.6982
Epoch 4/30
172/172 [==============================] - 150s 871ms/step - loss: 1.5480
Epoch 5/30
172/172 [==============================] - 150s 872ms/step - loss: 1.4593
Epoch 6/30
172/172 [==============================] - 150s 871ms/step - loss: 1.3992
Epoch 7/30
172/172 [==============================] - 150s 870ms/step - loss: 1.3532
Epoch 8/30
172/172 [==============================] - 149s 868ms/step - loss: 1.3142
Epoch 9/30
172/172 [==============================] - 149s 866ms/step - loss: 1.2783
Epoch 10/30
172/172 [==============================] - 149s 866ms/step - loss: 1.2462
Epoch 11/30
172/172 [==============================] - 149s 866ms/step - loss: 1.2139
Epoch 12/30
172/172 [==============================] - 149s 865

#### Rebuild model with single output for generating text one char at a time

In [0]:
tf.train.latest_checkpoint(checkpoint_dir)

'./model_output/seqGen/ckpt_30'

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [0]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the char returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted char as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [0]:
print(generate_text(model, start_string=u"ROMEO: "))

ROMEO: O, the news I'll be at dinner:
Sirrah, I pray how o'er-run he madest.

BRUTUS:
Then make you assist me, thou never trust
They'll take it off. Camillo,
And say yon our general, break out,
With raging winter, at our part, have taken
As shall deliver you.

CORIOLANUS:
Whoever once what then?
Have write in Saying, ho!

PETRUCHIO:
What, will you hear?
Why dost thou slip, nor now, what news?

BUSHY:
O, call them forgive; he's all deed sense you talk of death,
If whit is head, then he corrects contented: speak, worthy sir
Cisizenant comes our old and my life.

BENVOLIO:
A poor Hastings, people; peace,
Nurreverly look, behold, why she's a little thing.

DUKE OF AUMERLE:
For us, but than your eyes great dispersed,
Most bone, my answord' with rats that yet this co fingers, to your
wrongs to the people.

CORIOLANUS:
O villain, my tender barred chances
Our subjects, assistant and thee.

Second Senator:
So. they be not tongue-tied:
Or if wary barcal grant. They had guiltie is not so.

GLOUCE

In [0]:
print(generate_text(model, start_string=u"HAMLET: To be, or not to be: that is the question:"))

HAMLET: To be, or not to be: that is the question: when you
called, better.

POMPEY:
Very well, then,
But now 'tis pardon'd in the head of Milan are o
That I have begun,
And let his tribunes at the behalf conquered:
'llays us here anon.
Cousin of Buckingham?

BUCKINGHAM:
Most noble sir,
Le does remain in arms,
And stops my tale abadem.

MENENIUS:
Here, it;
Of the people, poor Milo!

DUKE OF YORK:

HENRY BOLINGBROKE:
What stoom me? They are not simple Henry,
Our trusty servants, stards, the sun comes
Some remotion.

LUCIO:
I warrant you, I must obey?

SAMPSON:
Fear
They ran a traitor, never bearing
married in evy
Know my daughter nor the time begins. Turns residence
come; and would prove unpurped thee take this vestard by
t.
Well they are but people,
He had thus stoop will be heredonable?
I will to her married to at the rest;
King Edward, Capulet'st; one one, it is request,
And then dwn you were dead! what then?

First Citizen:
We are blessed with the earth away.

BAPTISTA:
So shall the